In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
!mamba install -c conda-forge rdkit


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.1.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.1s
conda-forge/linux-64

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rdkit
from rdkit import Chem

import numpy as np 
import os
from rdkit.Chem import rdmolops
from scipy import sparse
import pandas as pd

from tqdm import tqdm
from rdkit.Chem.AllChem import EmbedMolecule
from rdkit.Chem import MolFromSmiles

from rdkit.Chem import AllChem as Chem
from rdkit.Chem import rdMolTransforms

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

from torch_geometric.data import Data
import torch.nn as nn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-fgwa

In [ ]:
import pandas as pd

import torch_geometric
from torch_geometric.data import Dataset, Data

import numpy as np 
import os


print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")


Torch version: 2.0.0+cu118
Cuda available: False
Torch geometric version: 2.4.0


In [ ]:
class Atom_Bond(Dataset):
    def __init__(self, filename, root, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(Atom_Bond, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        
        return 'not_implemented'

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        #featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            # Featurize molecule
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            #print(mol_obj)
            if(mol_obj):
              node_feats = self._get_node_features(mol_obj)
              edge_feats = self._get_edge_features(mol_obj)
              edge_index = self._get_adjacency_info(mol_obj)
              label = self._get_labels(mol["calc"])

              data = Data(x=node_feats, 
                          edge_index = edge_index,
                          edge_attr = edge_feats,
                          y = label,
                          smiles = mol["smiles"]
                          )
              torch.save(data,
                        os.path.join(self.processed_dir,
                                      f'data_{index}.pt'))

    def _get_node_features(self, mol):

        all_node_feats = []


        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
          #print(all_edge_feats)
        return torch.tensor(all_node_feats, dtype = torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        try:

          #mol3d = Chem.AddHs(mol)
          #mol3d = Chem.RemoveHs(mol3d)
          mol3d = mol
          Chem.EmbedMolecule(mol3d, randomSeed=42)
          Chem.MMFFOptimizeMolecule(mol3d)
        
          conf = mol3d.GetConformer() # atom1 = bond.GetBeginAtom()

          b_lengths = []

          for bond in mol3d.GetBonds():

              edge_feats = []
              atom1 = bond.GetBeginAtom()
              atom2 = bond.GetEndAtom()
              
              bl = rdMolTransforms.GetBondLength(mol3d.GetConformer(), atom1.GetIdx(), atom2.GetIdx())
              b_lengths = round(bl,2)

              # Feature 1: Bond type (as double)
              edge_feats.append(b_lengths)
              # Feature 2: Bond type (as double)
              edge_feats.append(float(bond.GetBondTypeAsDouble()))
              # Feature 3: Rings
              if bond.IsInRing():
                edge_feats.append(1.0)
              else:
                edge_feats.append(0.0)

              # Append node features to matrix
              all_edge_feats.append(edge_feats)

          all_edge_feats = np.asarray(all_edge_feats)
          #print(all_edge_feats)
          return torch.tensor(all_edge_feats, dtype = torch.float)

        except:
          return None


    def _get_adjacency_info(self, mol):
        try:
          adj_matrix = rdmolops.GetAdjacencyMatrix(mol)
          row, col = np.where(adj_matrix)
          coo = np.array(list(zip(row, col)))
          coo = np.reshape(coo, (2, -1))
          return torch.tensor(coo, dtype = torch.long)
        except:
          return None        

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def update_length(self):
        self.__len__ = len(self.data)

    def get(self, idx):
        """ Equivalent to __getitem__ in pytorch """
        data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))
        return data

Visualise dataset

In [ ]:
Data_f = '/content/SAMPL.csv'
data = pd.read_csv(Data_f)
data.head()

,iupac,smiles,expt,calc
0,"4-methoxy-N,N-dimethyl-benzamide",CN(C)C(=O)c1ccc(cc1)OC,-11.01,-9.625
1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,3-methylbut-1-ene,CC(C)C=C,1.83,2.452
3,2-ethylpyrazine,CCc1cnccn1,-5.45,-5.809
4,heptan-1-ol,CCCCCCCO,-4.21,-2.917


In [ ]:
print(data.shape)
print(data['CT_TOX'].value_counts())

Make Atom-Bond dataset

In [ ]:
# first [Errno 2] No such file or directory: '/content/data/raw/HIV.csv'
datasetAB = Atom_Bond('SAMPL.csv', root = '/content/data')

Processing...
100%|██████████| 642/642 [00:06<00:00, 102.26it/s]
Done!


In [ ]:
len(datasetAB)

print(datasetAB[0].edge_index.t())
print(datasetAB[0].x)
print(datasetAB[0].edge_attr)

tensor([[ 0,  6],
        [ 1,  7],
        [ 1,  7],
        [ 0,  6],
        [ 1,  7],
        [ 2,  8],
        [ 1,  8],
        [ 3,  7],
        [ 2,  8],
        [ 1,  9],
        [ 3,  8],
        [ 1, 11],
        [ 3,  9],
        [ 4,  8],
        [ 3,  9],
        [ 5, 10],
        [ 4, 10],
        [ 3,  5],
        [ 5, 10],
        [ 3,  9],
        [ 5, 11],
        [ 6,  8],
        [ 5, 11],
        [10, 12],
        [ 6, 12],
        [ 5, 11]])
tensor([[6., 1., 0., 4., 0.],
        [7., 3., 0., 3., 0.],
        [6., 1., 0., 4., 0.],
        [6., 3., 0., 3., 0.],
        [8., 1., 0., 3., 0.],
        [6., 3., 0., 3., 1.],
        [6., 2., 0., 3., 1.],
        [6., 2., 0., 3., 1.],
        [6., 3., 0., 3., 1.],
        [6., 2., 0., 3., 1.],
        [6., 2., 0., 3., 1.],
        [8., 2., 0., 3., 0.],
        [6., 1., 0., 4., 0.]])
tensor([[1.4400, 1.0000, 0.0000],
        [1.4400, 1.0000, 0.0000],
        [1.3900, 1.0000, 0.0000],
        [1.2300, 2.0000, 0.0000],
    

Save the formed dataset

In [ ]:
torch.save(datasetAB, 'datasetAB.pth')

In [ ]:
import pickle

# Assuming that your PyTorch dataset is called 'dataset'
with open('datasetAB_freesol.pkl', 'wb') as f:
    pickle.dump(datasetAB, f)

from google.colab import files
files.download('datasetAB_freesol.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#!zip -r /content/data/processed/
!zip -r /content/my_folder_freesol.zip /content/data/processed/

In [ ]:
from google.colab import files
files.download("/content/my_folder_freesol.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BA-A graphs

In [ ]:
class Bond_BondAngle(Dataset):
    def __init__(self, filename, root, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(Bond_BondAngle, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        
        return 'not_implemented'

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        #featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            # Featurize molecule
            #mol3d = Chem.MolFromSmiles(smiles)
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            if mol_obj:
              node_feats = self._get_node_features(mol_obj)
              edge_feats = self._get_edge_features(mol_obj)
              edge_index = self._get_adjacency_info(mol_obj)
              label = self._get_labels(mol["calc"])

            #print(edge_feats)

            data = Data(x=node_feats, 
                        edge_index = edge_index,
                        edge_attr = edge_feats,
                        y = label,
                        smiles = mol["smiles"]
                        )
            torch.save(data,
                       os.path.join(self.processed_dir,
                                    f'data_{index}.pt'))

    def _get_node_features(self, mol):

        all_node_feats = []

        try:

          #mol3d = Chem.AddHs(mol)
          #mol3d = Chem.RemoveHs(mol3d)
          mol3d = mol
          Chem.EmbedMolecule(mol3d, randomSeed=42)
          Chem.MMFFOptimizeMolecule(mol3d)
    
          conf = mol3d.GetConformer() # atom1 = bond.GetBeginAtom()

          for bond in mol3d.GetBonds():

              node_feats = []
              atom1 = bond.GetBeginAtom()
              atom2 = bond.GetEndAtom()

              # Feature 1: Atomic number        
              node_feats.append(atom1.GetAtomicNum())
              node_feats.append(atom2.GetAtomicNum())
              
              bl = rdMolTransforms.GetBondLength(mol3d.GetConformer(), atom1.GetIdx(), atom2.GetIdx())
              b_lengths = round(bl,2)

              # Feature 1: Bond type (as double)
              node_feats.append(b_lengths)
              # Feature 2: Bond type (as double)
              node_feats.append(float(bond.GetBondTypeAsDouble()))
              # Feature 3: Rings
              if bond.IsInRing():
                node_feats.append(1.0)
              else:
                node_feats.append(0.0)

              # Append node features to matrix
              all_node_feats.append(node_feats)

          all_node_feats = np.asarray(all_node_feats)
          return torch.tensor(all_node_feats, dtype = torch.float)

        except:
          all_node_feats = [1,1,1]
          all_node_feats = np.asarray(all_node_feats)
          return torch.tensor(all_node_feats, dtype = torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        try:

          #mol3d = Chem.AddHs(mol)
          #mol3d = Chem.RemoveHs(mol3d)
          #mol = MolFromSmiles('CCO')

          mol3d = mol
          Chem.EmbedMolecule(mol3d, randomSeed=42)
          Chem.MMFFOptimizeMolecule(mol3d)
          
          conf = mol3d.GetConformer() # atom1 = bond.GetBeginAtom()

          b_lengths = []

          # Compute the bond angles of all non-hydrogen bonds
          for bond in mol3d.GetBonds():

              edge_feats = []

              # Skip hydrogen bonds
              if bond.GetBeginAtom().GetAtomicNum() == 1 or bond.GetEndAtom().GetAtomicNum() == 1:
                  continue
              
              # Get the three atoms involved in the bond
              atom1 = bond.GetBeginAtom()
              atom2 = bond.GetEndAtom()
              atom3 = None

              for atom in atom1.GetNeighbors():
                  if atom.GetIdx() != atom2.GetIdx() and atom.GetAtomicNum() != 1:
                      atom3 = atom
                      bl = rdMolTransforms.GetBondLength(conf, atom1.GetIdx(), atom3.GetIdx())
                      flag1 = 1
                      break

              for atom in atom2.GetNeighbors():
                  if atom.GetIdx() != atom1.GetIdx() and atom.GetAtomicNum() != 1:
                      atom3 = atom
                      bl = rdMolTransforms.GetBondLength(conf, atom2.GetIdx(), atom3.GetIdx())
                      flag2 = 1
                      break

              angle = rdMolTransforms.GetAngleDeg(conf, atom1.GetIdx(), atom2.GetIdx(), atom3.GetIdx())
              edge_feats.append(angle)
              #di_angle = rdMolTransforms.GetDihedralDeg(conf, atom1.GetIdx(), atom2.GetIdx(), atom3.GetIdx())
              #edge_feats.append(di_angle)

              all_edge_feats.append(edge_feats)
              #print('edge_feat : ', all_edge_feats)

          all_edge_feats = np.asarray(all_edge_feats)
          return torch.tensor(all_edge_feats, dtype = torch.float)

        except:
          print('yy')
          all_edge_feats = np.asarray([0,0,0,0])
          return torch.tensor(all_edge_feats, dtype = torch.float)


    def _get_adjacency_info(self, mol):
      """ 
      This will return a matrix / 2d array of the shape
      [Number of edges, Edge Feature size]
      """
      try:

        mol3d = Chem.AddHs(mol)
        mol3d = Chem.RemoveHs(mol3d)
        Chem.EmbedMolecule(mol3d, randomSeed=42)
        Chem.MMFFOptimizeMolecule(mol3d)
      
        conf = mol3d.GetConformer()

        b_lengths = []
        ba_graph = []
        conf = mol.GetConformer()

        # Compute the bond angles of all non-hydrogen bonds
        for bond in mol.GetBonds():

            atom1 = bond.GetBeginAtom()
            atom2 = bond.GetEndAtom()

            if bond.GetBeginAtom().GetAtomicNum() == 1 or bond.GetEndAtom().GetAtomicNum() == 1:
                continue
            
            # Get the three atoms involved in the bond
            atom1 = bond.GetBeginAtom()
            atom2 = bond.GetEndAtom()
            atom3 = None

            ba1 = (atom1.GetIdx(), atom2.GetIdx())
            

            for atom in atom1.GetNeighbors():
                if atom.GetIdx() != atom2.GetIdx() and atom.GetAtomicNum() != 1:
                    atom3 = atom
                    bl = rdMolTransforms.GetBondLength(conf, atom1.GetIdx(), atom3.GetIdx())
                    flag1 = 1
                    #break

                    ba2 = (atom1.GetIdx(), atom3.GetIdx())
                    ba_edge = (ba1, ba2)
                    #print('ba_edge-',ba_edge)
                    ba_graph.append(ba_edge)
            
            for atom in atom2.GetNeighbors():
                if atom.GetIdx() != atom1.GetIdx() and atom.GetAtomicNum() != 1:
                    atom3 = atom
                    bl = rdMolTransforms.GetBondLength(conf, atom2.GetIdx(), atom3.GetIdx())
                    flag2 = 1
                    #break

                    ba2 = (atom2.GetIdx(), atom3.GetIdx())
                    ba_edge = (ba1, ba2)
                    #print('ba_edge*',ba_edge)
                    ba_graph.append(ba_edge)


            dic = {}
            count = 0
            edge_in = []

            for i in ba_graph:
              
              if tuple(sorted(i[0])) in dic:
                first_ind = dic[tuple(sorted(i[0]))]
              else:
                dic[tuple(sorted(i[0]))] = count
                first_ind = count
                count+=1

              
              if tuple(sorted(i[1])) in dic:
                second_ind = dic[tuple(sorted(i[1]))]
              else:
                dic[tuple(sorted(i[1]))] = count
                second_ind = count
                count+=1

              edge_i = [first_ind, second_ind]
              edge_in.append(edge_i)
              if not edge_in:
                print('hello')
                edge_in = [[1,0],[0,1]]

        return torch.tensor(edge_in, dtype = torch.long)
      
      except:
        print('xx')
        edge_in = [[1,2],[2,3],[0,1]]
        edge_in = torch.tensor(edge_in, dtype = torch.long)
        edge_in = edge_in.view(2,-1)
        return edge_in

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ Equivalent to __getitem__ in pytorch """
        data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))
        return data

In [ ]:
datasetBAB = Bond_BondAngle('SAMPL.csv',root = '/content/data_2')

print(datasetBAB[0].x)
print(datasetBAB[0].edge_attr)
print(datasetBAB[0].edge_index.t())

Processing...
  3%|▎         | 20/642 [00:00<00:10, 56.77it/s]

yy
xx


  7%|▋         | 48/642 [00:01<00:20, 28.38it/s]

yy
xx


  9%|▉         | 60/642 [00:01<00:15, 38.38it/s]

...

 96%|█████████▋| 619/642 [00:18<00:00, 55.63it/s]

yy


100%|██████████| 642/642 [00:18<00:00, 34.91it/s]

tensor([[6.0000, 7.0000, 1.4400, 1.0000, 0.0000],
        [7.0000, 6.0000, 1.4400, 1.0000, 0.0000],
        [7.0000, 6.0000, 1.3900, 1.0000, 0.0000],
        [6.0000, 8.0000, 1.2300, 2.0000, 0.0000],
        [6.0000, 6.0000, 1.4900, 1.0000, 0.0000],
        [6.0000, 6.0000, 1.4000, 1.5000, 1.0000],
        [6.0000, 6.0000, 1.3900, 1.5000, 1.0000],
        [6.0000, 6.0000, 1.3900, 1.5000, 1.0000],
        [6.0000, 6.0000, 1.3900, 1.5000, 1.0000],
        [6.0000, 6.0000, 1.3900, 1.5000, 1.0000],
        [6.0000, 8.0000, 1.3700, 1.0000, 0.0000],
        [8.0000, 6.0000, 1.4200, 1.0000, 0.0000],
        [6.0000, 6.0000, 1.3900, 1.5000, 1.0000]])
tensor([[115.6534],
        [ 32.1837],
        [121.0141],
        [ 31.3914],
        [117.5063],
        [120.6280],
        [120.1060],
        [119.5104],
        [120.1893],
        [120.6491],
        [111.4517],
        [ 33.4858],
        [123.5765]])
tensor([[ 0,  0,  1,  1,  2,  2,  2,  2,  3,  3,  4,  4,  4,  4,  5,  5,  5,  7,
       


Done!


In [ ]:
print(datasetBAB[221].x)
print(datasetBAB[221].edge_attr.shape)
print(datasetBAB[221].edge_index.t())

tensor([[6.0000, 6.0000, 1.5100, 1.0000, 0.0000],
        [6.0000, 6.0000, 1.5100, 1.0000, 0.0000],
        [6.0000, 6.0000, 1.5100, 1.0000, 0.0000],
        [6.0000, 8.0000, 1.4200, 1.0000, 0.0000],
        [8.0000, 6.0000, 1.4200, 1.0000, 0.0000]])
torch.Size([5, 1])
tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4],
        [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2, 4, 3]])


In [ ]:
print(datasetBAB[220].x)
print(datasetBAB[220].edge_attr.shape)
print(datasetBAB[220].edge_index.t())

tensor([[ 6.0000, 16.0000,  1.7700,  1.0000,  0.0000],
        [16.0000,  8.0000,  1.4500,  2.0000,  0.0000],
        [16.0000,  8.0000,  1.4500,  2.0000,  0.0000],
        [16.0000, 17.0000,  2.0500,  1.0000,  0.0000]])
torch.Size([4, 1])
tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
        [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2]])


In [ ]:
for idx,data in enumerate(datasetBAB):
  if data.edge_attr is None:
    print('xx')

In [ ]:
import pickle

# Assuming that your PyTorch dataset is called 'dataset'
with open('datasetBAB_freesolv.pkl', 'wb') as f:
    pickle.dump(datasetBAB, f)

from google.colab import files
files.download('datasetBAB_freesolv.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.save(datasetAB, 'datasetBAB.pth')
#!zip -r /content/data/processed/
!zip -r /content/my_folder_freesolBAB.zip /content/data_2/processed/
files.download("/content/my_folder_freesolBAB.zip")

updating: content/data_2/processed/ (stored 0%)
updating: content/data_2/processed/data_501.pt (deflated 60%)
updating: content/data_2/processed/data_639.pt (deflated 62%)
updating: content/data_2/processed/data_93.pt (deflated 62%)
updating: content/data_2/processed/data_444.pt (deflated 63%)
updating: content/data_2/processed/data_422.pt (deflated 66%)
updating: content/data_2/processed/data_608.pt (deflated 63%)
updating: content/data_2/processed/data_574.pt (deflated 62%)
updating: content/data_2/processed/data_24.pt (deflated 62%)
updating: content/data_2/processed/data_609.pt (deflated 68%)
updating: content/data_2/processed/data_261.pt (deflated 61%)
updating: content/data_2/processed/data_489.pt (deflated 67%)
updating: content/data_2/processed/data_77.pt (deflated 67%)
updating: content/data_2/processed/data_533.pt (deflated 62%)
updating: content/data_2/processed/data_25.pt (deflated 59%)
updating: content/data_2/processed/data_442.pt (deflated 59%)
updating: content/data_2/p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>